In [1]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms import *
from models import *
from dataloaders import *

from tabulate import tabulate

In [2]:
device = 'cpu'
dataset = AdultDataset(device=device)

In [3]:
def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test


def experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds):
    
    _, _, SGD = lr_fb_model_runner(dataset, SGD_hp, seeds)
    _, _, EI = lr_fb_model_runner(dataset, EI_hp, seeds)
    _, _, DP = lr_fb_model_runner(dataset, DP_hp, seeds)
    _, _, EO = lr_fb_model_runner(dataset, EO_hp, seeds)
    _, _, EODD = lr_fb_model_runner(dataset, EODD_hp, seeds)
    
    return SGD, EI, DP, EO, EODD

def hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

# SGD Hyperparameter Selection

In [4]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024
SGD_hp_test['fairness'] = ''
SGD_hp_test['delta_effort'] = 2.1

hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:19<00:00,  5.14epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:18<00:00,  5.42epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:19<00:00,  5.24epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:18<00:00,  5.55epochs/s]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei          dp          eo        eodd
---------------  ---------  ----------------  --------------  -----------  ----------  ----------  ----------
         0.0001          0          0.759625        0.759158  0.000638715  0.00732228  0.00195639  0.00195639
         0.001           0          0.837043        0.842533  0.0163369    0.119958    0.0720145   0.0720145
         0.01            0          0.846036        0.85082   0.0588518    0.14373     0.0879802   0.0879802
         0.1             0          0.844875        0.843362  0.0502871    0.102196    0.0600302   0.0600302


In [5]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.01
SGD_hp['lambda_'] = 0

# EI Hyperparameter Selection

In [6]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [1e-3, 1e-2]
EI_hp_test['lambda_'] = [1e-3, 1e-2, 0.1, 0.2, 0.5, 0.8]
EI_hp_test['batch_size'] = 128
EI_hp_test['fairness'] = 'EI'

hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:35<00:00,  2.79epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:35<00:00,  2.84epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:37<00:00,  2.68epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:37<00:00,  2.65epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei          dp           eo       eodd
---------------  ---------  ----------------  --------------  -----------  ----------  -----------  ---------
          0.001      0.001          0.84645         0.851152  0.0517852    0.128451    0.0753495    0.0753495
          0.001      0.01           0.846367        0.851649  0.0530106    0.119885    0.0345657    0.0482984
          0.001      0.1            0.845621        0.850489  0.0435915    0.110959    0.0242718    0.042112
          0.001      0.2            0.843964        0.851649  0.044072     0.10189     0.00668432   0.0361362
          0.001      0.5            0.819595        0.826952  0.0256201    0.0537745   0.0362218    0.0362218
          0.001      0.8            0.751295        0.750207  0            0           0            0
          0.01       0.001          0.847777        0.848831  0.0572313    0.132114    0.0846338

In [7]:
EI_hp = EI_hp_test.copy()
EI_hp['learning_rate'] = 0.001
EI_hp['lambda_'] = 0.5

# DP Hyperparameter Selection

In [8]:
DP_hp_test = SGD_hp_test.copy()
DP_hp_test['learning_rate'] = [1e-3, 5e-4]
DP_hp_test['lambda_'] = [1e-3, 1e-2, 0.1, 0.2, 0.5]
DP_hp_test['batch_size'] = 128
DP_hp_test['fairness'] = 'DP'

hyperparameter_test(dataset, DP_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:23<00:00,  4.21epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.06epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:23<00:00,  4.29epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:24<00:00,  4.14epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ---------  ---------
         0.001       0.001          0.846865        0.851649  0.0510907  0.129454    0.0773402  0.0773402
         0.001       0.01           0.846989        0.852147  0.0518501  0.127219    0.0538035  0.0538035
         0.001       0.1            0.823283        0.819824  0.0163855  0.10765     0.0608339  0.0608339
         0.001       0.2            0.659497        0.645284  0.117841   0.05405     0.0265657  0.0265657
         0.001       0.5            0.625347        0.617603  0.222017   0.00487641  0.0378349  0.0575579
         0.0005      0.001          0.846036        0.85082   0.0495105  0.128925    0.0700467  0.0700467
         0.0005      0.01           0.845787        0.852644  0.0525869  0.127195    0.0534746  0.0534746
         0.00

In [18]:
DP_hp = DP_hp_test.copy()
DP_hp['learning_rate'] = 0.001
DP_hp['lambda_'] = 0.1

# EO Hyperparameter Selection

In [10]:
EO_hp_test = SGD_hp_test.copy()
EO_hp_test['learning_rate'] = [1e-3, 1e-2]
EO_hp_test['lambda_'] = [0.001, 1e-2, 0.1, 0.2, 0.5, 0.8]
EO_hp_test['batch_size'] = 128
EO_hp_test['fairness'] = 'EO'

hyperparameter_test(dataset, EO_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.90epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.88epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.90epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:25<00:00,  3.86epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei          dp         eo       eodd
---------------  ---------  ----------------  --------------  -----------  ----------  ---------  ---------
          0.001      0.001          0.846658        0.851815  0.0509267    0.128783    0.0730299  0.0730299
          0.001      0.01           0.846202        0.850157  0.0491371    0.12523     0.0630821  0.0630821
          0.001      0.1            0.845373        0.851484  0.0456841    0.115471    0.0169954  0.0452658
          0.001      0.2            0.844668        0.850655  0.0446443    0.116599    0.0216404  0.0456275
          0.001      0.5            0.765386        0.766451  0.0150516    0.0158482   0.0378464  0.0378464
          0.001      0.8            0.751295        0.750207  0            0           0          0
          0.01       0.001          0.847569        0.849329  0.058585     0.129139    0.0786616  0.0786616
   

In [14]:
EO_hp = EO_hp_test.copy()
EO_hp['learning_rate'] = 0.001
EO_hp['lambda_'] = 0.1

# EOD Hyperparameter Selection

In [12]:
EODD_hp_test = SGD_hp_test.copy()
EODD_hp_test['learning_rate'] = [1e-3, 1e-2]
EODD_hp_test['lambda_'] = [1e-3, 1e-2, 0.1, 0.2, 0.5, 0.8]
EODD_hp_test['batch_size'] = 128
EODD_hp_test['fairness'] = 'EODD'

hyperparameter_test(dataset, EODD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:32<00:00,  3.07epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:32<00:00,  3.06epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:32<00:00,  3.04epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:32<00:00,  3.09epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei           dp          eo        eodd
---------------  ---------  ----------------  --------------  -----------  -----------  ----------  ----------
          0.001      0.001          0.846533        0.852147  0.0513999    0.128104     0.0657363   0.0657363
          0.001      0.01           0.84587         0.850489  0.0486101    0.122198     0.0421939   0.049805
          0.001      0.1            0.844337        0.850323  0.0432721    0.11611      0.0232965   0.0451856
          0.001      0.2            0.842306        0.849494  0.0466498    0.113418     0.0256161   0.042193
          0.001      0.5            0.751295        0.750207  0            0            0           0
          0.001      0.8            0.751295        0.750207  0            0            0           0
          0.01       0.001          0.847694        0.849163  0.0580357    0.128799     0.0786616   0.0

In [15]:
EODD_hp = EODD_hp_test.copy()
EODD_hp['learning_rate'] = 0.01
EODD_hp['lambda_'] = 0.1

# Model Training

In [20]:
seeds = np.arange(1,6)

SGD, EI, DP, EO, EODD = experiment_runner(dataset, SGD_hp, EI_hp, DP_hp, EO_hp, EODD_hp, seeds)

training seed 1 started


Training: 100%|██████████| 100/100 [00:16<00:00,  5.95epochs/s]


training seed 2 started


Training: 100%|██████████| 100/100 [00:16<00:00,  6.17epochs/s]


training seed 3 started


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:31<00:00,  3.18epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:30<00:00,  3.25epochs/s]

Training finished for all seeds.


In [21]:
result = []
models = ["SGD", "EI", "DP", "EO", "EODD"]
sol = [SGD, EI, DP, EO, EODD]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","EODD_mean","EODD_var"]))

model      accuracy_mean    accuracy_var    ei_mean      ei_var    dp_mean      dp_var     eo_mean      eo_var    EODD_mean    EODD_var
-------  ---------------  --------------  ---------  ----------  ---------  ----------  ----------  ----------  -----------  ----------
SGD             0.847968     0.000228481  0.0512867  0.00207924  0.122741   0.00298508  0.0968474   0.00511557   0.0968474   0.00511557
EI              0.818871     0.00274595   0.0214736  0.00554583  0.047252   0.00324576  0.00483895  0.00494637   0.00842311  0.00223752
DP              0.811514     0.00704084   0.023109   0.00467412  0.0995107  0.00167259  0.0630615   0.00588653   0.0630615   0.00588653
EO              0.84741      0.0011478    0.046095   0.00102533  0.110708   0.00382522  0.0320768   0.010279     0.0414772   0.00230791
EODD            0.842404     0.0021201    0.0407153  0.00553233  0.0971033  0.00796931  0.0170637   0.011484     0.03409     0.00406282
